<a href="https://colab.research.google.com/github/KaifAhmad1/Audio-Emotion-Analysis/blob/main/Audio_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Audio Data Preprocessing and Audio Sentiment and Emotion Analysis:

**Installing Necessary Dependencies:**

In [102]:
!pip install -qU \
     librosa \
     plotly

In [103]:
import os
import numpy as np
import pandas as pd
import librosa
import plotly.graph_objects as go
import plotly.io as pio
from skimage.io import imread

In [78]:
trainset_data = '/content/drive/MyDrive/audio-sentiment-data/TEST'
testset_data = '/content/drive/MyDrive/audio-sentiment-data/TRAIN'
file_path = '/content/drive/MyDrive/audio-sentiment-data/speech-emotion-data.csv'
speech_data = pd.read_csv(file_path)

In [79]:
trainset_data

'/content/drive/MyDrive/audio-sentiment-data/TEST'

In [80]:
testset_data

'/content/drive/MyDrive/audio-sentiment-data/TRAIN'

**Data Exploration:**

In [81]:
speech_data

,Filename,Class
0,346.wav,Negative
1,163.wav,Neutral
2,288.wav,Negative
3,279.wav,Negative
4,244.wav,Negative
...,...,...
245,204.wav,Neutral
246,46.wav,Positive
247,318.wav,Negative
248,49.wav,Positive


In [82]:
speech_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Filename  250 non-null    object
 1   Class     250 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [83]:
speech_data.describe()

,Filename,Class
count,250,250
unique,250,3
top,346.wav,Negative
freq,1,87


In [84]:
num_samples = speech_data.shape[0]
print(f"Dataset has {num_samples} samples")

print("Count of Positive and Negative samples:")
class_counts = speech_data['Class'].value_counts().reset_index()
print(class_counts)

Dataset has 250 samples
Count of Positive and Negative samples:
      Class  count
0  Negative     87
1  Positive     82
2   Neutral     81


**Audio Preprocessing:**

In [85]:
import librosa
# Set the sample rate for audio processing
sample_rate = 44100
def load_audio(file_path):
    # Load audio using librosa with specified parameters
    return librosa.load(file_path, sr=sample_rate, duration=2.5, offset=0.5)

In [105]:
def load_audio_features(path, avg_feat=0):
    features = []
    files = sorted(os.listdir(path))

    for fp in files:
        X, sr = load_audio(os.path.join(path, fp))
        f = librosa.feature.melspectrogram(y=X, sr=sample_rate)
        f = librosa.amplitude_to_db(f, ref=np.max)
        shapeY = f.shape[1]
        avg_feat = avg_feat if avg_feat != 0 else int((shapeY + avg_feat) / 2)
        feat_mat = np.zeros((1, f.shape[0], avg_feat))
        x_width = min(f.shape[1], avg_feat)
        feat_mat[0, :, :x_width] = f[:, :x_width]
        features.append(feat_mat)

    return np.vstack(features), files

In [106]:
# Set feature dimension
f_dim = 128

# Load and process training data
train_data, train_files = load_audio_features(trainset_data, f_dim)

# Load and process test data with the same feature dimension
test_data, test_files = load_audio_features(testset_data, train_data.shape[2])

In [107]:
# Print shapes
print("Train Shape:", train_data.shape, "| Test Shape:", test_data.shape)

Train Shape: (110, 128, 128) | Test Shape: (250, 128, 128)


In [108]:
# MelSpec to Images using Plotly
def save_img(f, fp):
    fig = go.Figure(data=go.Heatmap(z=f[::-1], colorscale='Viridis'))
    fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    pio.write_html(fig, fp)

In [109]:
def save_feature_to_image(path, save_dir, avg_feat=0):
    global sample_rate
    files = sorted(os.listdir(path))
    print("Scanning", path)

    for i, fp in enumerate(files):
        X, sr = load_audio(os.path.join(path, fp))
        f = librosa.feature.melspectrogram(y=X, sr=sample_rate)
        f = librosa.amplitude_to_db(f, ref=np.max)
        img = np.zeros((f.shape[0], avg_feat))
        x_width = min(f.shape[1], avg_feat)
        img[:, :x_width] = f[:, :x_width]
        fname = os.path.join(save_dir, fp.split('.')[0] + '.html')
        save_img(img, fname)

In [110]:
# Set directories for training and test images
train_img_dir = '/content/train_images'
test_img_dir = '/content/test_images'

In [111]:
# Create training image directory if not exists and save Mel spectrogram images
if not os.path.exists(train_img_dir):
    os.mkdir(train_img_dir)
    save_feature_to_image(trainset_data, train_img_dir, f_dim)

In [112]:
# Create test image directory if not exists and save Mel spectrogram images using the same feature dimension as training data
if not os.path.exists(test_img_dir):
    os.mkdir(test_img_dir)
    save_feature_to_image(testset_data, test_img_dir, train_data.shape[2])

In [113]:
# Scan image features from a directory
def scan_img_features(path):
    features = []
    files = sorted(os.listdir(path))

    for file_name in files:
        file_path = os.path.join(path, file_name)
        try:
            img = imread(file_path)[:,:,:3] / 255.0
            features.append(img)
        except Exception as e:
            print(f"Skipping {file_name} due to error: {e}")

    return np.array(features), files

In [115]:
if os.path.exists(train_img_dir):
    train_data_img, train_files_img = scan_img_features(train_img_dir)
    for img in train_data_img:
        fig = go.Figure(data=go.Heatmap(z=img[::-1], colorscale='Viridis'))
        fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                          yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
        fig.show()

if os.path.exists(test_img_dir):
    test_data_img, test_files_img = scan_img_features(test_img_dir)
    for img in test_data_img:
        fig = go.Figure(data=go.Heatmap(z=img[::-1], colorscale='Viridis'))
        fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                          yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
        fig.show()

Skipping 112.html due to error: Could not find a backend to open `/content/train_images/112.html`` with iomode `ri`.
Skipping 113.html due to error: Could not find a backend to open `/content/train_images/113.html`` with iomode `ri`.
Skipping 115.html due to error: Could not find a backend to open `/content/train_images/115.html`` with iomode `ri`.
Skipping 119.html due to error: Could not find a backend to open `/content/train_images/119.html`` with iomode `ri`.
Skipping 123.html due to error: Could not find a backend to open `/content/train_images/123.html`` with iomode `ri`.
Skipping 124.html due to error: Could not find a backend to open `/content/train_images/124.html`` with iomode `ri`.
Skipping 125.html due to error: Could not find a backend to open `/content/train_images/125.html`` with iomode `ri`.
Skipping 128.html due to error: Could not find a backend to open `/content/train_images/128.html`` with iomode `ri`.
Skipping 13.html due to error: Could not find a backend to open 